In [ ]:
# !pip install opencv-python
# !pip install pinecone-client
# !pip install opencv-python-headless

In [8]:
import pinecone
import os
from pinecone import Pinecone, ServerlessSpec

import openai
import numpy as np

# from google.colab.patches import cv2_imshow
import urllib.request
import cv2

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

OPENAI_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Initialize the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY, environment='us-east-1')

In [5]:
index_name = 'description-embed'
description_index = pc.Index(index_name)
description_index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 776976}},
 'total_vector_count': 776976}

In [6]:
# create connection with images index
index_name2 = 'clip-embed'
clip_index = pc.Index(index_name2)
clip_index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 664537}},
 'total_vector_count': 664537}

In [7]:
query_text = "I am attending a soireee. Help me find a red dress and then explain to me which dress I should buy."

In [ ]:
from openai import OpenAI
client = OpenAI()

In [9]:
# Create an embedding for the query
# response = openai.Embedding.create(input=query_text, model="text-embedding-3-large")
# query_embedding = response['data'][0]['embedding']

response = client.embeddings.create(
    input=query_text,
    model="text-embedding-3-large"
)

print(response.data[0].embedding)

query_embedding = response.data[0].embedding

[-0.04118664935231209, 0.0033528595231473446, -0.017385462298989296, 0.034226734191179276, 0.008019653148949146, -0.021209118887782097, 0.02060764469206333, 0.00776187889277935, -0.023271316662430763, 0.048031996935606, -0.04024147614836693, -0.013568967580795288, 0.01593906059861183, -0.008141380734741688, 0.00899346824735403, -0.00600399961695075, -0.0006014740210957825, 0.011757384985685349, -0.0178294088691473, -0.012752681039273739, -0.02202540636062622, 0.015824494883418083, -0.009065072983503342, 0.014879321679472923, -0.018445203080773354, 0.012394661083817482, 0.004833273123949766, 0.012158367782831192, -0.02889939397573471, 0.02821199595928192, 0.04367846995592117, 0.019633829593658447, 0.05098208412528038, -0.0620950311422348, -0.060720235109329224, -0.007045838516205549, 0.04725867137312889, 0.04591251537203789, 0.03803607076406479, 0.014213403686881065, 0.0006542819901369512, 0.014299328438937664, -0.016483252868056297, -0.021681705489754677, 0.010024567134678364, -0.01184

In [10]:
# Perform the query
query_response = description_index.query(
    vector=query_embedding,  # The embedding vector
    top_k=5  # Number of top results to retrieve
)

In [11]:
# Print simple retrieval
print(query_response)

{'matches': [{'id': 'B07KVYJLL3', 'score': 0.4719477, 'values': []},
             {'id': 'B0B6VGZGQM', 'score': 0.451321542, 'values': []},
             {'id': 'B01175GAAA', 'score': 0.445820361, 'values': []},
             {'id': 'B099QWT18P', 'score': 0.442919612, 'values': []},
             {'id': 'B09DBTGV83', 'score': 0.442606658, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 39}}


In [13]:
# Print metadata
for match in query_response.matches:
    vector = description_index.fetch(ids=[match.id])
    # print(vector)
    print(match.score)
    print(vector['vectors'][match.id]['id'])
    print(vector['vectors'][match.id]['metadata']['title'])
    print(vector['vectors'][match.id]['metadata']['store'])
    print(vector['vectors'][match.id]['metadata']['average_rating'])
    print(vector['vectors'][match.id]['metadata']['review'])
    break

0.4719477
B07KVYJLL3
Formal Prom Party Dress Off Long Sleeves Fall Fashion Bodycon Work Midi Pencil Dress Sexy V Neck Clubwear S Red
HUUSA
1.7
0 Upvoted: 'Nothing like the picture orange red in color.  This item was to large in the chest and I'm a 32 D. No wire support as shown in the picture. Very dissatisfied!!!'


#### WRITE MY LANGCHAIN AGENT

In [ ]:
# !pip install pinecone-client openai langchain
# !pip install --upgrade openai
# !pip install --upgrade langchain_openai

In [20]:
# Function to query Pinecone index
def query_pinecone(query: str):
    # Embed the query using an embedding model
    # Assuming you have an embedding function, e.g., using OpenAI's embeddings

    # Create an embedding for the query
    response = client.embeddings.create(
    input=query_text,
    model="text-embedding-3-large"
    )
    query_embedding = response.data[0].embedding

    # Query the Pinecone index
    response = description_index.query(
        vector=query_embedding,
        top_k=5,  # Number of top matches to return
        include_metadata=True
    )

    # Process the response to extract product information
    results = response['matches']
    products = [match['metadata'] for match in results]
    return products

# Integrate this function with LangChain's tool
from langchain.agents import tool

@tool
def search_product(query: str) -> str:
    """Search product in Pinecone database by name"""
    return query_pinecone(query)

In [21]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

sys_prompt = """
You are an online shopping assistant. Your job is to help users find products, provide product details, and answer questions.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sys_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_product],
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_product],
    verbose=True
)

In [22]:
agent_executor.invoke({"input": "I am attending a soireee. Help me find a red dress and then explain to me by I should or should not buy the products you suggeest.",})



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'query': 'red dress'}`


[{'average_rating': 1.7, 'review': "0 Upvoted: 'Nothing like the picture orange red in color.  This item was to large in the chest and I'm a 32 D. No wire support as shown in the picture. Very dissatisfied!!!'", 'store': 'HUUSA', 'title': 'Formal Prom Party Dress Off Long Sleeves Fall Fashion Bodycon Work Midi Pencil Dress Sexy V Neck Clubwear S Red'}, {'average_rating': 4.5, 'review': "10 Upvoted: '[[VIDEOID:bdb72c82b31ed849b72b3cbff6217ebb]] This dress definitely makes me look fat and I’m not 😂 I would definitely size down, I was swimming in this dress even though my measurements are exact to the size I ordered.'", 'store': 'DIRASS', 'title': "DIRASS Party Dress for Women Velvet Dress Red Bridesmaid Burgundy Women's Cocktail Dresses with Sleeves Long Maxi Wrap Dresses for Wedding Guest L,Wine Red"}, {'average_rating': 4.0, 'review': "0 Upvoted: 'Cheaply made.  I will let my daughter we

{'input': 'I am attending a soireee. Help me find a red dress and then explain to me by I should or should not buy the products you suggeest.',
 'output': 'Here are the details for the two red dresses you were interested in:\n\n1. **DIRASS Party Dress for Women Velvet Dress Red Bridesmaid Burgundy Women\'s Cocktail Dresses with Sleeves Long Maxi Wrap Dresses for Wedding Guest L, Wine Red**\n   - Store: DIRASS\n   - Average Rating: 4.5\n   - Review: "This dress definitely makes me look fat and I’m not 😂 I would definitely size down, I was swimming in this dress even though my measurements are exact to the size I ordered."\n\n2. **Deardresses Women\'s One-shoulder Chiffon Short Bridesmaid Prom Dress (S, Red)**\n   - Store: Deardresses\n   - Average Rating: 4.0\n   - Review: "Cheaply made. I will let my daughter wear to the dance."\n\nBased on the reviews, the DIRASS dress has a high rating but one review mentions sizing issues. The Deardresses dress has a good rating but one review criti

#### MODIFIED AGENT
1. Extract and Summarize Reviews: Parse the reviews and ratings, summarize the reviews, and calculate the average rating.
2. Determine Product Quality: Based on the average rating and review sentiment, determine whether the product is generally good or not.

In [27]:
# Function to create an embedding
def create_embedding(query_text):
    response = client.embeddings.create(
    input=query_text,
    model="text-embedding-3-large"
    )
    query_embedding = response.data[0].embedding
    return query_embedding

# Function to query Pinecone index
def query_pinecone(query: str):
    query_embedding = create_embedding(query)
    response = description_index.query(
        vector=query_embedding,
        top_k=5,
        include_metadata=True
    )
    results = response['matches']
    products = [match['metadata'] for match in results]
    return products

# Function to summarize reviews and determine product quality
def summarize_reviews(products):
    summary = []
    total_rating = 0
    num_reviews = len(products)
    
    for product in products:
        title = product.get('title', 'Unknown Title')
        store = product.get('store', 'Unknown Store')
        average_rating = product.get('average_rating', 0)
        review = product.get('review', 'No review available')
        
        summary.append(f"Title: {title}\nStore: {store}\nAverage Rating: {average_rating}\nReview: {review}\n")
        total_rating += average_rating
    
    average_rating_overall = total_rating / num_reviews if num_reviews else 0
    quality_assessment = "good" if average_rating_overall >= 4 else "not good"
    
    summary_text = "\n\n".join(summary)
    overall_summary = (
        f"Overall, these products have an average rating of {average_rating_overall:.2f}, "
        f"which is considered {quality_assessment}.\n\n"
        f"Based on the reviews:\n"
        f"{summary_text}\n\n"
        f"My advice is based on the average rating and specific reviews. For example, products with higher ratings like '{products[1].get('title', 'Unknown Title')}' "
        f"are generally well-received for their fit and quality, while products with lower ratings such as '{products[0].get('title', 'Unknown Title')}' "
        f"are noted for their poor quality and mismatched descriptions."
    )
    
    return overall_summary


@tool
def search_product(query: str) -> str:
    """Search for a product in the Pinecone database and summarize the reviews and overall quality."""
    products = query_pinecone(query)
    return summarize_reviews(products)

sys_prompt = """
You are an online shopping assistant. Your job is to help users find products, provide product details, summarize reviews, and assess overall product quality.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sys_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_product], 
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=[search_product], 
    verbose=True
)

# Function to perform the query
def perform_query(query: str):
    chat_history = []
    response = agent_executor({"input": query, "chat_history": chat_history})
    return response

In [28]:
# Example query
query = "I am attending a soiree. Help me find a red dress and then explain to me why I should or should not buy the products you suggest."
response = perform_query(query)

# Print the response
print(response)



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'query': 'red dress'}`


Overall, these products have an average rating of 3.20, which is considered not good.

Based on the reviews:
Title: IHOT Women Dress Red
Store: IHOT
Average Rating: 2.6
Review: 1 Upvoted: 'If you have any sort of a chest, do not order this dress. I got the largest size and there was no way I was getting everything in it. Disappointed. I will continue to order from OTEN, Zaful and OWIN instead.'


Title: IHOT Women Dress Red
Store: IHOT
Average Rating: 3.0
Review: 2 Upvoted: 'The quality of the dress itself was okay, but the chest was way too big (and I am a 32D) and the drop waist was not flattering, looked like I was a little kid playing dress up.'


Title: IHOT Women Dress Red
Store: IHOT
Average Rating: 3.0
Review: 0 Upvoted: 'The dress is cute, but the fabric is a little chintzy. It works for what I need it for.'


Title: Seeing Red Bubbly Bae Includes Dress, Hat
Store: Seeing Red
Av

In [1]:
# Get related url
import pandas as pd
full_data = pd.read_parquet('/Users/ez/Downloads/amazon_fashion_clean_051624.parquet')

In [12]:
full_data[full_data['title'] == 'DRESSIS Double Layered Chiffon Mini Dress RED L']['images'].iloc[0]

array([{'hi_res': 'https://m.media-amazon.com/images/I/61E94v3TLgL._AC_UL1500_.jpg', 'large': 'https://m.media-amazon.com/images/I/412xA6v+WhL._AC_.jpg', 'thumb': 'https://m.media-amazon.com/images/I/412xA6v+WhL._AC_SR38,50_.jpg', 'variant': 'MAIN'},
       {'hi_res': 'https://m.media-amazon.com/images/I/61hZr1nHSdL._AC_UL1500_.jpg', 'large': 'https://m.media-amazon.com/images/I/41k+pQUxJUL._AC_.jpg', 'thumb': 'https://m.media-amazon.com/images/I/41k+pQUxJUL._AC_SR38,50_.jpg', 'variant': 'PT01'},
       {'hi_res': 'https://m.media-amazon.com/images/I/61JF9ne8UWL._AC_UL1500_.jpg', 'large': 'https://m.media-amazon.com/images/I/31wqQrE00NL._AC_.jpg', 'thumb': 'https://m.media-amazon.com/images/I/31wqQrE00NL._AC_SR38,50_.jpg', 'variant': 'PT02'},
       {'hi_res': 'https://m.media-amazon.com/images/I/71CkWG9voGL._AC_UL1500_.jpg', 'large': 'https://m.media-amazon.com/images/I/417vvH8AjEL._AC_.jpg', 'thumb': 'https://m.media-amazon.com/images/I/417vvH8AjEL._AC_SR38,50_.jpg', 'variant': 'PT03

In [9]:
# Function to filter out only the "large" images
def filter_large_images(images):
    return [image['large'] for image in images if image.get('variant') == 'MAIN']

# Extract the images for the specified title
matching_images = full_data[full_data['title'] == 'DRESSIS Double Layered Chiffon Mini Dress RED L']['images'].iloc[0]

# Filter out only the "large" images
large_images = filter_large_images(matching_images)

In [10]:
product_main_image = 'https://m.media-amazon.com/images/I/412xA6v+WhL._AC_.jpg'